# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json, os
from pathlib import Path

irp_client = IRPClient()

### Configurations

In [57]:
edm_name = "CBHU" # ensure unique within IRP
portfolio_name = "CBHU_2" # ensure unique within the edm
accounts_file_name = "Modeling_202503_Moodys_CBHU_Account.csv"
locations_file_name = "Modeling_202503_Moodys_CBHU_Location.csv"
analysis_job_name = "CBHU_Analysis"
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"

### Create EDM

In [39]:
print(json.dumps(irp_client.edm.create_edm(edm_name, "databridge-1"), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22790500
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Create Portfolio

In [58]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name, portfolio_name)
portfolio_id = portfolio_response['id'].split('/')[-1]
print(json.dumps(portfolio_response, indent=2))

{
  "id": "3"
}


### MRI Import

In [63]:
notebook_dir = Path(os.getcwd())
working_files_dir = notebook_dir / "workspace" / "workflows" / "_Tools" / "files" / "working_files"

bucket_response = irp_client.mri_import.create_aws_bucket()
bucket_url = bucket_response.headers['location']
bucket_id = bucket_url.split('/')[-1]

accounts_file_path = os.path.join(working_files_dir, accounts_file_name)
accounts_credentials = irp_client.mri_import.get_file_credentials(bucket_url, accounts_file_name, 100, "account")
irp_client.mri_import.upload_file_to_s3(accounts_credentials, accounts_file_path)

locations_file_path = os.path.join(working_files_dir, locations_file_name)
locations_credentials = irp_client.mri_import.get_file_credentials(bucket_url, locations_file_name, 100, "location")
irp_client.mri_import.upload_file_to_s3(locations_credentials, locations_file_path)

mapping_file_name = "mapping.json"
mapping_file_path = os.path.join(working_files_dir, mapping_file_name)
mapping_file_id = irp_client.mri_import.upload_mapping_file(mapping_file_path, bucket_id).json()

import_response = irp_client.mri_import.execute_mri_import(
    edm_name,
    int(portfolio_id),
    int(bucket_id),
    accounts_credentials['file_id'],
    locations_credentials['file_id'],
    mapping_file_id
)
print(json.dumps(import_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: RUNNING; Percent complete: 100
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793657
Workflow status: FINISHED; Percent complete: 100
{
  "id": 22793657,
  "userName": "Partner - Premium

### Create Treaty

In [51]:
treaty_name = "test_treaty"
treaty_type_name = "Working Excess"
currency_name = "US Dollar"
attachment_basis = "Losses Occurring"
attachment_level = "Location"

cedant_data = irp_client.edm.get_cedants_by_edm(edm_name)["searchItems"]
lob_data = irp_client.edm.get_lobs_by_edm(edm_name)["searchItems"]

treaty_type_data = irp_client.treaty.get_treaty_types_by_edm(edm_name)["entityItems"]["values"]
treaty_type_match = next(
    (item for item in treaty_type_data if item["name"] == treaty_type_name),
    None  # fallback if no match found
)

attachment_basis_data = irp_client.treaty.get_treaty_attachment_bases_by_edm(edm_name)["entityItems"]["values"]
attachment_basis_match = next(
    (item for item in attachment_basis_data if item["name"] == attachment_basis),
    None  # fallback if no match found
)

attachment_level_data = irp_client.treaty.get_treaty_attachment_levels_by_edm(edm_name)["entityItems"]["values"]
attachment_level_match = next(
    (item for item in attachment_level_data if item["name"] == attachment_level),
    None  # fallback if no match found
)

currency_data = irp_client.reference_data.get_currencies()["entityItems"]["values"]
currency_match = next(
    (item for item in currency_data if item["name"] == currency_name),
    None  # fallback if no match found
)

if len(cedant_data) > 0 and len(lob_data) > 0 and treaty_type_match and attachment_basis_match and attachment_level_match and currency_match:
    data = {
        "treatyNumber": treaty_name,
        "treatyName": treaty_name,
        "treatyType": treaty_type_match,
        "riskLimit": 3000000,
        "occurLimit": 9000000,
        "attachPt": 2000000,
        "cedant": cedant_data[0],
        "effectDate": "2025-10-15T17:49:10.637Z",
        "expireDate": "2026-10-15T17:49:10.637Z",
        "currency": {'code': currency_match['code'], 'name': currency_match['name']},
        "attachBasis": attachment_basis_match,
        "attachLevel": attachment_level_match,
        "pcntCovered": 100,
        "pcntPlaced": 95,
        "pcntRiShare": 100,
        "pcntRetent": 100,
        "premium": 0,
        "numOfReinst": 99,
        "reinstCharge": 0,
        "aggregateLimit": 0,
        "aggregateDeductible": 0,
        "priority": 1,
        "retentAmt": "",
        "isValid": True,
        "userId1": "",
        "userId2": "",
        "maolAmount": "",
        "lobs": lob_data,
        "tagIds": []
    }

    # print(json.dumps(data, indent=2))

    treaty_response = irp_client.treaty.create_treaty(edm_name, data)
    lob_ids = []
    for lob in lob_data:
        lob_ids.append(lob['id'])
    print(json.dumps(irp_client.treaty.assign_lobs(edm_name, treaty_response['id'], lob_ids), indent=2))
    

[
  {
    "context": {
      "headers": {},
      "entityAnnotations": [],
      "entityStream": {
        "closed": false,
        "committed": false
      },
      "length": -1,
      "acceptableMediaTypes": [
        {
          "type": "*",
          "subtype": "*",
          "parameters": {},
          "quality": 1000,
          "wildcardType": true,
          "wildcardSubtype": true
        }
      ],
      "acceptableLanguages": [
        "*"
      ],
      "allowedMethods": [],
      "links": [],
      "stringHeaders": {},
      "committed": false,
      "requestCookies": {},
      "lengthLong": -1,
      "responseCookies": {}
    },
    "status": 201,
    "length": -1,
    "statusInfo": "CREATED",
    "metadata": {},
    "allowedMethods": [],
    "cookies": {},
    "links": [],
    "stringHeaders": {},
    "headers": {}
  }
]


### Upgrade EDM Version

In [52]:
print(json.dumps(irp_client.edm.upgrade_edm_version(edm_name), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22792168
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22792168
Workflow status: RUNNING; Percent complete: 70
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22792168
Workflow status: FINISHED; Percent complete: 100
{
  "id": 22792168,
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submitTime": "2025-10-16T19:47:02.555Z",
  "startTime": "2025-10-16T19:47:04Z",
  "endTime": "2025-10-16T19:47:16Z",
  "name": "CBHU_Hybrid",
  "type": "EDM_DATA_UPGRADE",
  "progress": 100,
  "priority": "medium",
  "entitlement": "RI-RISKMODELER",
  "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
  "jobs": [
    {
      "id": "8ffedd1a-e72d-4532-8304-22e2b5634acf",
      "taskId": 1,
      "workflowId": 22792168,
      "status": "Succeeded",
      "submitTime": "2025-10

### GeoHaz

In [64]:
print(json.dumps(
    irp_client.portfolio.geohaz_portfolio(edm_name, 
                                          portfolio_id, 
                                          geocode=True, 
                                          hazard_eq=True, 
                                          hazard_ws=True), 
    indent=2
))

[
  {
    "name": "geocode",
    "type": "geocode",
    "engineType": "RL",
    "version": "22.0",
    "layerOptions": {
      "aggregateTriggerEnabled": "true",
      "geoLicenseType": "0",
      "skipPrevGeocoded": false
    }
  },
  {
    "name": "earthquake",
    "type": "hazard",
    "engineType": "RL",
    "version": "22.0",
    "layerOptions": {
      "overrideUserDef": false,
      "skipPrevHazard": false
    }
  },
  {
    "name": "windstorm",
    "type": "hazard",
    "engineType": "RL",
    "version": "22.0",
    "layerOptions": {
      "overrideUserDef": false,
      "skipPrevHazard": false
    }
  }
]
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793682
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793682
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793682
Wor

### Analysis

In [65]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name,
    [treaty_response['id']]
)
print(json.dumps(analysis_response, indent=2))

{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU",
  "eventRateSchemeId": 739,
  "exposureType": "PORTFOLIO",
  "id": "3",
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    "1"
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown": true,
    "numMaxLossEvent": 1
  },
  "jobName": "CBHU_Analysis"
}
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Wor

### Group Analyses

In [43]:
group_name = "Test Analysis Group"
analysis_names = []
analysis_ids = [33181, 33182]
print(json.dumps(irp_client.analysis.create_analysis_group(analysis_ids, group_name), indent=2))

{
  "analysisIds": [
    33181,
    33182
  ],
  "name": "Test Analysis Group",
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "simulateToPLT": true,
  "numOfSimulations": 50000,
  "propagateDetailedLosses": false,
  "reportingWindowStart": "01/01/2021",
  "simulationWindowStart": "01/01/2021",
  "simulationWindowEnd": "12/31/2021"
}
Submitting workflow request...


HTTPError: 400 Client Error: Bad Request for url: https://api-euw1.rms-ppe.com/riskmodeler/v2/analysis-groups  | server: {'code': 'CLIENT-400', 'message': 'Unable to read JSON', 'logId': '0d4239151bbc0525'}